# Métodos de gradiente conjugado no lineal 
## Daniel Vallejo Aldana
### Optimización 1

## Fletcher-Reeves

In [1]:
import numpy as np
from numpy import linalg as la 


def backtracking(f, g, xk, dk, fk, gk, alpha0, c1, c2, max_iter=1000):
    alpha = 0
    beta = float('inf')
    alphai = alpha0
    it = 0
    while 1 and it < max_iter:
        if f(xk + alphai*dk) > fk + c1*alphai*np.dot(np.transpose(gk), dk):
            beta = alphai
            alphai = (alpha + beta)/2
        elif abs(np.dot(np.transpose(g(xk + alphai*dk)), dk)) < c2*np.dot(np.transpose(gk), dk):
            alpha = alphai
            if beta == float('inf'):
                alphai = 2*alpha
            else:
                alphai = (alpha + beta)/2
        else:
            break
        it+=1
    return alphai


def CG_FR(x0,tf,grad,maxiter=10000,tolgrad=1e-4,tolf=1e-4):
    g_k=grad(x0)
    dk=-g_k
    xk=x0
    iterations=0
    while iterations<maxiter and la.norm(g_k)>tolgrad:
        fk=tf(xk)
        alpha=backtracking(tf,grad,xk,dk,fk,g_k,1.0,0.2,0.44)
        xk=xk+alpha*dk
        gtemp=g_k
        g_k=grad(xk)
        beta=np.dot(g_k.T,g_k)/np.dot(gtemp.T,gtemp)
        dk=-g_k+beta*dk
        iterations+=1
        print(iterations,"&","{:.4f}".format(la.norm(g_k)),"&","{:.4f}".format(tf(xk)))
    return np.array([iterations, xk, la.norm(g_k),tf(xk)])

## Modificación de Polak-Ribiere

In [2]:
def CG_PR(x0,tf,grad,maxiter=1000,tolgrad=1e-4,tolf=1e-4):
    g_k=grad(x0)
    dk=-g_k
    xk=x0
    iterations=0
    while iterations<maxiter and la.norm(g_k)>tolgrad:
        fk=tf(xk)
        alpha=backtracking(tf,grad,xk,dk,fk,g_k,1.0,0.2,0.44)
        xk=xk+alpha*dk
        gtemp=g_k
        g_k=grad(xk)
        beta=max(0,np.dot(g_k.T,(g_k-gtemp))/np.dot(gtemp.T,gtemp))
        dk=-g_k+beta*dk
        iterations+=1
        
        print(iterations,"&","{:.4f}".format(la.norm(g_k)),"&","{:.4f}".format(tf(xk)))
    return np.array([iterations, xk, la.norm(g_k),tf(xk)])

## Hestenes-Stiefel

In [179]:
def CG_HS(x0,tf,grad,maxiter=1000,tolgrad=1e-4,tolf=1e-4):
    g_k=grad(x0)
    dk=-g_k
    xk=x0
    iterations=0
    while iterations<maxiter and la.norm(g_k)>tolgrad:
        fk=tf(xk)
        alpha=backtracking(tf,grad,xk,dk,fk,g_k,1.0,0.2,0.44)
        xk=xk+alpha*dk
        gtemp=g_k
        g_k=grad(xk)
        beta=max(0,np.dot(g_k.T,(g_k-gtemp))/np.dot((g_k-gtemp).T,dk))
        dk=-g_k+beta*dk
        iterations+=1
        print(iterations,"&","{:.4f}".format(la.norm(g_k)),"&","{:.4f}".format(tf(xk)))
    return np.array([iterations, xk, la.norm(g_k),tf(xk)])

## Rosenbrock n=100

In [180]:
#Definamos la función que vamos a usar 
def rosen(x): #La función de rosembruck en general
    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

def r_der(x):
        xm = x[1:-1]
        xm_m1 = x[:-2]
        xm_p1 = x[2:]
        der =np.zeros(x.shape)
        der[1:-1] = 200*(xm-xm_m1**2) - 400*(xm_p1 - xm**2)*xm - 2*(1-xm)
        der[0] = -400*x[0]*(x[1]-x[0]**2) - 2*(1-x[0])
        der[-1] = 200*(x[-1]-x[-2]**2)
        return der

def rosen_hess(x):
        H = np.diag(-400*x[:-1],1) - np.diag(400*x[:-1],-1)
        diagonal = np.zeros(len(x))
        diagonal[0] = 1200*x[0]-400*x[1]+2
        diagonal[-1] = 200
        diagonal[1:-1] = 202 + 1200*x[1:-1]**2 - 400*x[2:]
        H = H + np.diag(diagonal)
        return H

p_inicial=np.ones(100)
p_inicial[0]=-1.2
p_inicial[98]=-1.2

In [181]:
res_fr=CG_FR(p_inicial,rosen,r_der) #Fletcher Reeves

1 & 421.2740 & 178.6237
2 & 198.1919 & 102.4540
3 & 153.1186 & 61.5932
4 & 144.5405 & 38.5439
5 & 102.8113 & 16.7409
6 & 91.5695 & 12.0248
7 & 65.4809 & 11.1311
8 & 42.4425 & 9.9730
9 & 47.4197 & 9.0701
10 & 46.2728 & 8.2307
11 & 35.7361 & 7.7441
12 & 23.8931 & 7.3617
13 & 16.9116 & 7.1837
14 & 18.0161 & 7.1137
15 & 18.2676 & 7.0986
16 & 14.4686 & 7.0068
17 & 11.9536 & 6.9439
18 & 9.5549 & 6.8982
19 & 8.0926 & 6.8682
20 & 10.2255 & 6.8466
21 & 10.5109 & 6.8393
22 & 8.5864 & 6.8127
23 & 8.1012 & 6.7984
24 & 8.1402 & 6.7844
25 & 8.2030 & 6.7676
26 & 8.7414 & 6.7469
27 & 12.4330 & 6.7173
28 & 15.4383 & 6.7071
29 & 17.6191 & 6.6874
30 & 18.0730 & 6.6578
31 & 18.8640 & 6.5753
32 & 21.3548 & 6.5248
33 & 22.0707 & 6.4919
34 & 21.5104 & 6.4343
35 & 21.0422 & 6.3747
36 & 21.2812 & 6.3228
37 & 22.0769 & 6.2827
38 & 22.1501 & 6.2551
39 & 22.1521 & 6.2010
40 & 22.3816 & 6.1788
41 & 23.6005 & 6.1503
42 & 23.9444 & 6.1424
43 & 23.8182 & 6.1005
44 & 23.3916 & 6.0517
45 & 24.1297 & 6.0181
46 & 24.2373

In [167]:
res_pr=CG_PR(p_inicial,rosen,r_der) #Polak-Ribiere

1 & 421.2740 & 178.6237
2 & 352.3678 & 126.5078
3 & 262.7827 & 60.7525
4 & 215.8480 & 35.9814
5 & 131.7354 & 31.0211
6 & 111.4165 & 17.5714
7 & 70.9790 & 12.2149
8 & 51.3662 & 9.5764
9 & 50.4096 & 9.5215
10 & 16.7182 & 8.1557
11 & 16.8910 & 7.9813
12 & 15.6874 & 7.9283
13 & 7.1978 & 7.7991
14 & 7.8322 & 7.7747
15 & 8.1912 & 7.7740
16 & 4.8261 & 7.7463
17 & 4.9345 & 7.7298
18 & 6.4497 & 7.7086
19 & 11.4335 & 7.6627
20 & 26.9306 & 7.5737
21 & 25.1740 & 7.1638
22 & 10.7586 & 6.8539
23 & 9.8085 & 6.7765
24 & 8.6748 & 6.7437
25 & 3.9092 & 6.7004
26 & 4.5884 & 6.6870
27 & 8.2370 & 6.6693
28 & 14.1651 & 6.5970
29 & 28.9645 & 6.3749
30 & 33.6609 & 6.2942
31 & 17.3430 & 5.8535
32 & 16.5909 & 5.7328
33 & 11.6008 & 5.6190
34 & 9.4973 & 5.5236
35 & 9.1451 & 5.4987
36 & 8.3220 & 5.4524
37 & 8.0095 & 5.4474
38 & 3.7073 & 5.4164
39 & 4.0829 & 5.4065
40 & 11.6088 & 5.3798
41 & 17.9845 & 5.1622
42 & 17.1823 & 5.0076
43 & 17.2412 & 5.0075
44 & 19.4176 & 4.8549
45 & 18.8364 & 4.8228
46 & 8.9031 & 4.6976


In [168]:
res_hs=CG_HS(p_inicial,rosen,r_der) #Hestens-Stiefel

1 & 421.2740 & 178.6237
2 & 352.1916 & 121.9507
3 & 258.0141 & 60.8745
4 & 127.0916 & 30.9042
5 & 101.5793 & 16.0914
6 & 90.6857 & 10.8917
7 & 48.5741 & 9.0885
8 & 21.8269 & 7.8725
9 & 15.3314 & 7.4051
10 & 7.2093 & 7.2763
11 & 5.7721 & 7.2168
12 & 5.8490 & 7.1801
13 & 5.0569 & 7.1437
14 & 4.6385 & 7.1145
15 & 9.7158 & 7.0932
16 & 9.3444 & 7.0268
17 & 19.1279 & 6.9343
18 & 18.5211 & 6.8071
19 & 16.8966 & 6.5247
20 & 18.1889 & 6.3418
21 & 15.3885 & 6.1362
22 & 14.6223 & 5.9257
23 & 12.9454 & 5.7892
24 & 11.3612 & 5.6648
25 & 10.8992 & 5.5546
26 & 9.7416 & 5.4679
27 & 9.3531 & 5.3786
28 & 9.0336 & 5.2853
29 & 7.8422 & 5.2068
30 & 8.0489 & 5.1463
31 & 7.0934 & 5.0738
32 & 6.6739 & 5.0118
33 & 6.1302 & 4.9548
34 & 11.2815 & 4.9041
35 & 18.9755 & 4.8351
36 & 15.4936 & 4.6975
37 & 14.7621 & 4.5742
38 & 9.0075 & 4.5272
39 & 3.2885 & 4.4832
40 & 2.9645 & 4.4757
41 & 2.8750 & 4.4725
42 & 4.0656 & 4.4623
43 & 4.1898 & 4.4473
44 & 7.0187 & 4.4231
45 & 12.7201 & 4.3898
46 & 11.9268 & 4.3409
47 & 1

In [169]:
print(res_fr[0],"&",res_fr[2],"&",res_fr[3])

1556 & 9.910847082334586e-05 & 3.986623854672206


In [170]:
print(res_pr[0],"&",res_pr[2],"&",res_pr[3])

260 & 7.785321601282875e-05 & 3.986623855134566


In [171]:
print(res_hs[0],"&",res_hs[2],"&",res_hs[3])

158 & 8.870700726745377e-05 & 3.986623854305516


In [172]:
p_inicial=np.ones(100)   #Cambiemos el punto inicial 
p_inicial[0]=1.2
p_inicial[98]=-1.2

In [173]:
res_fr=CG_FR(p_inicial,rosen,r_der) #Fletcher Reeves

1 & 421.0110 & 174.5062
2 & 198.1272 & 98.4433
3 & 153.0356 & 57.5901
4 & 144.5189 & 34.5448
5 & 102.7974 & 12.7401
6 & 91.8445 & 8.0514
7 & 65.6895 & 7.1641
8 & 42.6007 & 6.0071
9 & 47.6526 & 5.1029
10 & 46.4964 & 4.2577
11 & 35.9162 & 3.7668
12 & 23.9930 & 3.3805
13 & 16.9811 & 3.2008
14 & 18.0684 & 3.1301
15 & 18.3112 & 3.1150
16 & 14.5116 & 3.0228
17 & 12.0054 & 2.9595
18 & 9.5726 & 2.9135
19 & 8.0995 & 2.8835
20 & 10.2457 & 2.8619
21 & 10.5265 & 2.8545
22 & 8.5869 & 2.8279
23 & 8.1058 & 2.8136
24 & 8.1401 & 2.7997
25 & 8.1978 & 2.7829
26 & 8.7356 & 2.7622
27 & 12.4224 & 2.7327
28 & 15.4243 & 2.7224
29 & 17.5989 & 2.7026
30 & 18.0387 & 2.6728
31 & 18.7930 & 2.5900
32 & 21.2596 & 2.5389
33 & 21.9572 & 2.5049
34 & 21.3571 & 2.4456
35 & 20.8238 & 2.3835
36 & 20.9833 & 2.3283
37 & 21.6815 & 2.2830
38 & 22.7920 & 2.2565
39 & 23.0829 & 2.2402
40 & 22.6718 & 2.2049
41 & 22.9856 & 2.1613
42 & 23.2449 & 2.1515
43 & 24.7200 & 2.1205
44 & 25.1455 & 2.1163
45 & 25.0285 & 2.0866
46 & 24.4791 & 

In [174]:
res_pr=CG_PR(p_inicial,rosen,r_der) #Polak-Ribiere

1 & 421.0110 & 174.5062
2 & 352.1219 & 122.4350
3 & 262.6173 & 56.6184
4 & 215.9095 & 31.9106
5 & 131.8779 & 27.0709
6 & 111.8003 & 13.6133
7 & 71.2784 & 8.2525
8 & 51.4918 & 5.5961
9 & 50.5751 & 5.5453
10 & 16.7464 & 4.1707
11 & 16.7691 & 3.9952
12 & 15.5313 & 3.9416
13 & 7.0500 & 3.8140
14 & 7.6232 & 3.7893
15 & 7.8926 & 3.7878
16 & 4.6628 & 3.7618
17 & 4.7938 & 3.7453
18 & 6.1290 & 3.7238
19 & 10.9412 & 3.6770
20 & 25.4290 & 3.5779
21 & 23.1574 & 3.1634
22 & 9.3613 & 2.8910
23 & 8.7535 & 2.8284
24 & 7.8806 & 2.8036
25 & 3.6715 & 2.7682
26 & 4.2113 & 2.7553
27 & 7.5152 & 2.7287
28 & 21.8790 & 2.6221
29 & 25.0291 & 2.4602
30 & 27.5601 & 2.1295
31 & 25.1640 & 2.0781
32 & 13.3762 & 1.8157
33 & 13.4675 & 1.7111
34 & 7.3607 & 1.6221
35 & 7.2587 & 1.5889
36 & 6.8496 & 1.5823
37 & 3.7156 & 1.5598
38 & 4.0990 & 1.5481
39 & 12.4876 & 1.5153
40 & 21.9340 & 1.3555
41 & 16.3913 & 1.0620
42 & 13.8620 & 0.8684
43 & 10.3622 & 0.8159
44 & 8.0603 & 0.7625
45 & 7.7691 & 0.7613
46 & 2.8951 & 0.7341
47 

In [175]:
res_hs=CG_HS(p_inicial,rosen,r_der) #Hestens-Stiefel

1 & 421.0110 & 174.5062
2 & 351.9746 & 117.8826
3 & 257.8213 & 56.7454
4 & 126.8419 & 26.8079
5 & 101.4730 & 12.0680
6 & 90.8366 & 6.9053
7 & 48.5505 & 5.1012
8 & 21.8057 & 3.8888
9 & 15.4575 & 3.4230
10 & 7.2766 & 3.2920
11 & 5.8151 & 3.2316
12 & 5.8712 & 3.1944
13 & 5.0599 & 3.1577
14 & 4.6423 & 3.1286
15 & 9.7699 & 3.1084
16 & 9.3899 & 3.0428
17 & 19.3204 & 2.9538
18 & 18.7381 & 2.8301
19 & 17.1256 & 2.5443
20 & 18.4892 & 2.3617
21 & 15.7059 & 2.1569
22 & 14.9612 & 1.9422
23 & 13.3058 & 1.8038
24 & 11.6629 & 1.6754
25 & 11.2340 & 1.5623
26 & 10.0635 & 1.4747
27 & 9.7015 & 1.3840
28 & 9.4426 & 1.2886
29 & 8.2029 & 1.2074
30 & 8.4527 & 1.1454
31 & 7.4465 & 1.0709
32 & 7.0327 & 1.0073
33 & 6.4899 & 0.9494
34 & 11.9859 & 0.9029
35 & 11.4697 & 0.8027
36 & 9.6534 & 0.6584
37 & 9.1522 & 0.5839
38 & 8.2150 & 0.5270
39 & 7.6897 & 0.4700
40 & 6.7754 & 0.4247
41 & 6.8629 & 0.3929
42 & 5.8306 & 0.3500
43 & 5.9848 & 0.3258
44 & 5.3331 & 0.3047
45 & 5.4060 & 0.2837
46 & 5.0139 & 0.2635
47 & 4.793

In [176]:
print(res_fr[0],"&",res_fr[2],"&",res_fr[3])
print(res_pr[0],"&",res_pr[2],"&",res_pr[3])
print(res_hs[0],"&",res_hs[2],"&",res_hs[3])

1028 & 9.834015478183303e-05 & 2.118990606843819e-10
277 & 8.907078041226747e-05 & 2.007414721898093e-09
145 & 8.319525416091396e-05 & 1.433562873196919e-10


## Prueba 2. Función wood

In [154]:
def wood(datos): #Función general, target function
    x1,x2,x3,x4=datos
    return 100*((x2-x1**2)**2)+(1-x1)**2+(1-x3)**2+90*((x4-x3**2)**2)+10.1*((x2-1)**2+(x4-1)**2)+19.8*(x2-1)*(x4-1)

def wood_derivate(x):
    g=np.zeros(len(x))
    g[0]=2.0*(200.0*x[0]**3-200*x[0]*x[1]+x[0]-1)
    g[1]=-200*x[0]**2+220.2*x[1]+19.8*x[3]-40
    g[2]=2*(180*x[2]**3-180*x[3]*x[2]+x[2]-1)
    g[3]=19.8*x[1]-180*x[2]**2+200.2*x[3]-40
    return g

p_inicial=[-3,-1,-3,-1]


In [160]:
res_fr=CG_FR(p_inicial,wood,wood_derivate) #Fletcher Reeves

1 & 258.7141 & 160.2761
2 & 8.2509 & 35.1420
3 & 8.1599 & 35.0125
4 & 138.7531 & 31.4089
5 & 138.7531 & 31.4089
6 & 138.7531 & 31.4089
7 & 138.7531 & 31.4089
8 & 138.7531 & 31.4089
9 & 138.7531 & 31.4089
10 & 138.7531 & 31.4089
11 & 138.7531 & 31.4089
12 & 138.7531 & 31.4089
13 & 138.7531 & 31.4089
14 & 147.2362 & 31.1588
15 & 164.9959 & 30.3765
16 & 187.9924 & 29.8448
17 & 202.4907 & 29.6634
18 & 217.3001 & 29.3298
19 & 231.9136 & 28.8974
20 & 245.5074 & 28.4568
21 & 256.8266 & 28.1315
22 & 259.8596 & 27.8023
23 & 258.3878 & 26.8500
24 & 248.7888 & 26.1900
25 & 230.4599 & 25.7941
26 & 217.5505 & 25.4770
27 & 184.5669 & 24.7304
28 & 152.6635 & 24.3738
29 & 124.6317 & 24.2210
30 & 112.9342 & 24.1281
31 & 90.0954 & 23.9357
32 & 73.7482 & 23.8631
33 & 62.0741 & 23.8323
34 & 53.9032 & 23.8160
35 & 48.4382 & 23.8039
36 & 45.1142 & 23.7916
37 & 43.5215 & 23.7765
38 & 43.3583 & 23.7565
39 & 44.3964 & 23.7285
40 & 49.6384 & 23.6951
41 & 51.6814 & 23.6840
42 & 60.9566 & 23.6169
43 & 67.2875 & 2

In [156]:
res_pr=CG_PR(p_inicial,wood,wood_derivate) #Polak-Ribiere

1 & 3063.3315 & 2261.4021
2 & 1229.3338 & 734.8549
3 & 493.2486 & 245.6652
4 & 200.0799 & 87.3800
5 & 63.4848 & 38.1865
6 & 32.6702 & 32.3592
7 & 21.5654 & 29.6235
8 & 29.6395 & 21.5797
9 & 32.0316 & 13.6088
10 & 24.3595 & 10.0927
11 & 15.9954 & 8.1035
12 & 3.3033 & 7.9581
13 & 2.6229 & 7.9470
14 & 2.2971 & 7.8830
15 & 0.5000 & 7.8791
16 & 0.4430 & 7.8789
17 & 0.4401 & 7.8771
18 & 0.1055 & 7.8770
19 & 0.0293 & 7.8770
20 & 0.0558 & 7.8770
21 & 0.0163 & 7.8769
22 & 0.0239 & 7.8769
23 & 0.0170 & 7.8769
24 & 0.0065 & 7.8769
25 & 0.0026 & 7.8769
26 & 0.0070 & 7.8769
27 & 0.0063 & 7.8769
28 & 0.0048 & 7.8769
29 & 0.0066 & 7.8769
30 & 0.0202 & 7.8769
31 & 0.0562 & 7.8769
32 & 0.0366 & 7.8769
33 & 0.3993 & 7.8765
34 & 0.9072 & 7.8763
35 & 1.0131 & 7.8763
36 & 1.1768 & 7.8721
37 & 6.0138 & 7.7850
38 & 6.8595 & 7.7654
39 & 4.6720 & 7.7520
40 & 2.7046 & 7.7393
41 & 1.6568 & 7.7295
42 & 1.7629 & 7.7252
43 & 2.1421 & 7.7131
44 & 3.3431 & 7.6967
45 & 3.2033 & 7.6877
46 & 1.1937 & 7.6843
47 & 0.6903 

In [157]:
res_hs=CG_HS(p_inicial,wood,wood_derivate) #Hestens-Stiefel

1 & 3063.3315 & 2261.4021
2 & 1229.3338 & 734.8549
3 & 493.2486 & 245.6652
4 & 200.0799 & 87.3800
5 & 63.4848 & 38.1865
6 & 32.6702 & 32.3592
7 & 21.5654 & 29.6235
8 & 107.5421 & 21.3187
9 & 119.3092 & 18.9658
10 & 34.1589 & 9.8121
11 & 11.2197 & 9.0893
12 & 8.6698 & 8.9312
13 & 6.5598 & 8.0094
14 & 4.0254 & 7.9515
15 & 1.7463 & 7.8796
16 & 0.3671 & 7.8776
17 & 0.2099 & 7.8775
18 & 0.5301 & 7.8772
19 & 0.0813 & 7.8768
20 & 0.0230 & 7.8768
21 & 0.0321 & 7.8768
22 & 0.0071 & 7.8768
23 & 0.0145 & 7.8768
24 & 0.0104 & 7.8768
25 & 0.0289 & 7.8768
26 & 1.3078 & 7.8759
27 & 1.4310 & 7.8758
28 & 1.9043 & 7.8712
29 & 0.9323 & 7.8675
30 & 1.4675 & 7.8651
31 & 3.9039 & 7.8477
32 & 4.3630 & 7.8420
33 & 2.5449 & 7.8286
34 & 1.1185 & 7.8237
35 & 0.3528 & 7.8224
36 & 0.8082 & 7.8218
37 & 0.2032 & 7.8198
38 & 0.4976 & 7.8195
39 & 0.4019 & 7.8185
40 & 0.3854 & 7.8177
41 & 4.3139 & 7.8086
42 & 5.6349 & 7.7985
43 & 4.4218 & 7.7790
44 & 4.6495 & 7.3510
45 & 4.8366 & 7.3271
46 & 2.2629 & 7.3080
47 & 1.9041

In [161]:
print(res_fr[0],"&",res_fr[2],"&",res_fr[3])
print(res_pr[0],"&",res_pr[2],"&",res_pr[3])
print(res_hs[0],"&",res_hs[2],"&",res_hs[3])

6410 & 9.888474672337977e-05 & 4.0348627226313136e-11
188 & 5.464592531393034e-05 & 6.779933745429497e-11
158 & 1.612187209829625e-05 & 5.8017373025995055e-11


In [162]:
p_incial=[1.2,1.2,1.2,1.2]
res_fr=CG_FR(p_inicial,wood,wood_derivate) #Fletcher Reeves

1 & 258.7141 & 160.2761
2 & 8.2509 & 35.1420
3 & 8.1599 & 35.0125
4 & 138.7531 & 31.4089
5 & 138.7531 & 31.4089
6 & 138.7531 & 31.4089
7 & 138.7531 & 31.4089
8 & 138.7531 & 31.4089
9 & 138.7531 & 31.4089
10 & 138.7531 & 31.4089
11 & 138.7531 & 31.4089
12 & 138.7531 & 31.4089
13 & 138.7531 & 31.4089
14 & 147.2362 & 31.1588
15 & 164.9959 & 30.3765
16 & 187.9924 & 29.8448
17 & 202.4907 & 29.6634
18 & 217.3001 & 29.3298
19 & 231.9136 & 28.8974
20 & 245.5074 & 28.4568
21 & 256.8266 & 28.1315
22 & 259.8596 & 27.8023
23 & 258.3878 & 26.8500
24 & 248.7888 & 26.1900
25 & 230.4599 & 25.7941
26 & 217.5505 & 25.4770
27 & 184.5669 & 24.7304
28 & 152.6635 & 24.3738
29 & 124.6317 & 24.2210
30 & 112.9342 & 24.1281
31 & 90.0954 & 23.9357
32 & 73.7482 & 23.8631
33 & 62.0741 & 23.8323
34 & 53.9032 & 23.8160
35 & 48.4382 & 23.8039
36 & 45.1142 & 23.7916
37 & 43.5215 & 23.7765
38 & 43.3583 & 23.7565
39 & 44.3964 & 23.7285
40 & 49.6384 & 23.6951
41 & 51.6814 & 23.6840
42 & 60.9566 & 23.6169
43 & 67.2875 & 2

In [139]:
res_pr=CG_PR(p_inicial,wood,wood_derivate) #Polak-Ribiere

1 & 3063.3315 & 2261.4021
2 & 1229.3338 & 734.8549
3 & 493.2486 & 245.6652
4 & 200.0799 & 87.3800
5 & 63.4848 & 38.1865
6 & 32.6702 & 32.3592
7 & 21.5654 & 29.6235
8 & 59.7136 & 20.2480
9 & 83.8644 & 18.8801
10 & 25.7271 & 10.4534
11 & 12.8449 & 9.9951
12 & 15.3998 & 9.6264
13 & 9.4966 & 8.2483
14 & 3.5771 & 8.0612
15 & 4.9404 & 7.9632
16 & 1.6357 & 7.9127
17 & 3.2560 & 7.8955
18 & 1.6866 & 7.8793
19 & 0.2765 & 7.8777
20 & 0.2486 & 7.8776
21 & 0.3376 & 7.8773
22 & 0.1019 & 7.8770
23 & 0.1585 & 7.8770
24 & 0.0343 & 7.8769
25 & 0.0873 & 7.8769
26 & 0.0086 & 7.8769
27 & 0.0048 & 7.8769
28 & 0.0208 & 7.8769
29 & 0.0292 & 7.8769
30 & 0.0214 & 7.8769
31 & 0.0126 & 7.8769
32 & 0.0275 & 7.8769
33 & 0.0066 & 7.8769
34 & 0.0113 & 7.8769
35 & 0.0059 & 7.8769
36 & 0.0089 & 7.8769
37 & 0.0279 & 7.8769
38 & 0.0107 & 7.8769
39 & 0.0144 & 7.8769
40 & 0.0208 & 7.8769
41 & 0.0095 & 7.8769
42 & 0.0051 & 7.8769
43 & 0.0144 & 7.8769
44 & 0.0250 & 7.8769
45 & 0.0396 & 7.8769
46 & 0.0248 & 7.8769
47 & 0.0387

In [140]:
res_hs=CG_HS(p_inicial,wood,wood_derivate) #Hestens-Stiefel

1 & 3063.3315 & 2261.4021
2 & 1229.3338 & 734.8549
3 & 493.2486 & 245.6652
4 & 200.0799 & 87.3800
5 & 63.4848 & 38.1865
6 & 32.6702 & 32.3592
7 & 21.5654 & 29.6235
8 & 107.5421 & 21.3187
9 & 119.3092 & 18.9658
10 & 34.1589 & 9.8121
11 & 11.2197 & 9.0893
12 & 8.6698 & 8.9312
13 & 6.5598 & 8.0094
14 & 4.0254 & 7.9515
15 & 1.7463 & 7.8796
16 & 0.3671 & 7.8776
17 & 0.2099 & 7.8775
18 & 0.5301 & 7.8772
19 & 0.0813 & 7.8768
20 & 0.0230 & 7.8768
21 & 0.0321 & 7.8768
22 & 0.0071 & 7.8768
23 & 0.0145 & 7.8768
24 & 0.0104 & 7.8768
25 & 0.0289 & 7.8768
26 & 1.3078 & 7.8759
27 & 1.4310 & 7.8758
28 & 1.9043 & 7.8712
29 & 0.9323 & 7.8675
30 & 1.4675 & 7.8651
31 & 3.9039 & 7.8477
32 & 4.3630 & 7.8420
33 & 2.5449 & 7.8286
34 & 1.1185 & 7.8237
35 & 0.3528 & 7.8224
36 & 0.8082 & 7.8218
37 & 0.2032 & 7.8198
38 & 0.4976 & 7.8195
39 & 0.4019 & 7.8185
40 & 0.3854 & 7.8177
41 & 4.3139 & 7.8086
42 & 5.6349 & 7.7985
43 & 4.4218 & 7.7790
44 & 6.2319 & 7.2228
45 & 4.1616 & 7.2112
46 & 4.1308 & 7.1872
47 & 3.0488